# ÉCOLE IVADO/MILA EN APPRENTISSAGE PROFOND
# SESSION D' AUTOMNE 2018 
# Tutoriel : Réseaux à convolutions

## Auteurs: 

Margaux Luck <margaux.luck@rd.mila.quebec>

Jeremy Pinto <jeremy.pinto@rd.mila.quebec>

Mathieu Germain <mathieu.germain@rd.mila.quebec>

## Préface

Ce tutoriel à pour but d'introduire les concepts fondamentaux sur les réseaux de neurones à convolution à l'aide d'exemples concrets.

# Initialisation

Avant de commencer, nous devons nous assurer d'installer les librairies nécessaires pour le tutoriel à l'aide de `pip`.  Pour se faire, exécutez la cellufle suivante en la sélectionnant et en cliquant `shift`+`Enter`. Ceci peut prendre quelques minutes.

In [ ]:
!pip3 install torch torchvision Pillow matplotlib

Afin de vous assurer que l'installation s'est bien faite, importez toutes les libraries et modules dont nous nous servirons pour ce tutoriel en exécutant la prochaine cellule:

In [ ]:
import torch

use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if use_gpu else "cpu")

print("Torch version: ", torch.__version__)
print("GPU Available: {}".format(use_gpu))

# Le jeu de données MNIST
MNIST est le **jeu de données de référence de classification** utilisé en **vision par ordinateur**. Il est hébergé sur le <a href="http://yann.lecun.com/exdb/mnist/">le site de Yann LeCun</a>. Il se compose d'**images de chiffres manuscripts**. Quelques exemples sont données ci-dessous :

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/mnist.png?raw=true)

Il inclut également des **étiquettes de classes pour chaque image**, indiquant à quel chiffre elle correspond. Par exemple, les étiquettes des images ci-dessus sont 5, 0, 4 et 1.

Il se compose de **60 000 exemples d'entraînement** et de **10 000 exemples de test**. Les images sont toutes de la même taille (**28x28 pixels**). Chaque pixel est représenté par un chiffre entre 0 et 255 indiquant une nuance de gris. En fonction des modèles que nous allons tester les images seront utilisées telles quelles ou bien aplaties.

Les classes sont balancées, c'est à dire qu'il y a une proportion plus ou moins égale des différents chiffres dans notre jeu de données.

## Objectif

Nous cherchons à concevoir un algorithme pour classifier ces chiffres correctement. Dans notre cas, nous utiliserons comme entrée l'intensité de tous les pixels d'une image et essayerons de prédir la valeur du chiffre en question. Ce problème peut se résumer à:

`f(image) = chiffre prédit`

où `f` est une fonction quelconque. 

Dans ce tutoriel, nous considérerons un **perceptron multi-couche** ainsi qu'un **réseau de neuronnes à convolution**. Ces fonctions prennent en entrée l'intensité des pixels et effectuent des opérations mathématiques sur ces valeurs. La valeur de retour de la fonction est un vecteur de taille *1xN* où chaque entrée correspond à la probabilité que le chiffre en question soit ce chiffre. La somme de toutes ces valeurs est nécessairement de 1. Notre prédiction finale est l'entrée pour laquelle la probabilité est maximale. Par exemple, la prédiction

`[0.8, 0.1, 0, 0, 0, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0]`

indique à 80% de chances qu'il s'agit du chiffre 0.

Pour chacun de ces modèles, nous chercherons à "apprendre" la meilleure solution. Pour se faire, nous commencerons à partir d'une solution aléatoire et chercherons à converger vers une solution optimale. À chaque itération, nous comparons les résultats de notre prédiction à la réelle valeure et ajustons nos modèles par la suite jusqu'à ce que nous satisfaisons un certain critère d'arret

## Télécharger les données et créer le chargeur de données

Avant de commencer, nous devons nous assurer d'avoir accès aux données de MNIST. Nous utiliserons les fonctions natives à PyTorch pour les récupérer.

### Boîte à outils
**Rappel:** Dans PyTorch, il existe des fonctions pour charger, mélanger et augmenter les données. 

Une façon simple de charger les données dans PyTorch est : 
<ul>
<li>D'utiliser une classe enfant de <a href="http://pytorch.org/docs/master/data.html#torch.utils.data.Dataset">`torch.utils.data.Dataset`</a> où les méthodes `__getitem__` et `__len__` sont à compléter.</li>
<li>D'utiliser la classe <a href="http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader">`torch.utils.data.DataLoader`</a> pour lire et mettre en mémoire les données.</li>
</ul>


Par chance, dans pytorch, il existe déjà une classe enfant de Dataset pour utiliser MNIST : <a href="http://pytorch.org/docs/master/torchvision/datasets.html#mnist">`torchvision.datasets.MNIST`</a>.

<a href="http://pytorch.org/docs/master/torchvision/datasets.html">D'autres jeux de données sont aussi disponibles.</a>

**Remarque:** <a href="http://pytorch.org/docs/master/tensors.html#torch.Tensor.view">`torch.Tensor.view()`</a> renvoie un nouveau tenseur avec les mêmes données que le tenseur d'origine mais avec une taille différente. Cela peut donc être utilisé pour aplatir une image, par exemple.

In [ ]:
import numpy as np
import random
import torch
from torch.utils.data import sampler, DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms


manualSeed = 1234
use_gpu = torch.cuda.is_available()

# Fixing random seed
random.seed(manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)
if use_gpu:
   torch.cuda.manual_seed_all(manualSeed)

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset.
    From: https://github.com/pytorch/vision/issues/168
    
    Parameters
    ----------
    num_samples: int
      # of desired datapoints
    start: int
      Offset where we should start selecting from
    """
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples


train_dataset = MNIST(root='../data', 
                      train=True, 
                      transform=transforms.ToTensor(),  
                      download=True)

test_dataset = MNIST(root='../data', 
                     train=False, 
                     transform=transforms.ToTensor())

train_dataset_sizes = len(train_dataset)
num_train_samples = int(0.8 * train_dataset_sizes)
num_valid_samples = train_dataset_sizes - num_train_samples
num_test_samples = len(test_dataset)

print('# of train examples: {}'.format(num_train_samples))
print('# of valid examples: {}'.format(num_valid_samples))
print('# of test examples: {}'.format(num_test_samples))

batch_size = 100

train_loader = DataLoader(dataset=train_dataset,
                          sampler=ChunkSampler(num_train_samples, 0),
                          batch_size=batch_size, 
                          shuffle=False)

valid_loader = DataLoader(dataset=train_dataset,
                          sampler=ChunkSampler(
                              num_valid_samples, num_train_samples),
                          batch_size=batch_size, 
                          shuffle=False)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)


Visualisons les données d'entraînement !

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

inputs, classes = next(iter(train_loader))

print('Inputs size: {}'.format(inputs.size()))
print('Classes size: {}'.format(classes.size()))

# Random image of the batch
img1 = 255 - inputs[np.random.randint(len(inputs))] * 255

# Plot the image
print('\n\nDisplay the first image:')
img1 = img1.numpy()[0, :, :]
plt.imshow(img1, cmap='gray', vmin=0, vmax=255)
plt.grid(False)
plt.show()


# CPU ou GPU
**Rappel:** <a href="http://pytorch.org/docs/master/cuda.html#module-torch.cuda">`torch.cuda`</a> est une librairie qui permet d'utiliser des GPUs pour effectuer les calculs sur des tenseurs. La librairie inclus des tenseurs de type CUDA qui ont les mêmes fonctions que les tenseurs réguliers mais qui utilisent des GPUs pour leurs calculs, au lieu d'un CPU. <a href="http://pytorch.org/docs/master/cuda.html#torch.cuda.is_available">`torch.cuda.is_available()`</a> retourne un booléen indiquant si CUDA est présentement disponible. Pour passer d'un tenseur de type CPU à un tenseur de type GPU, il suffit de lui ajouter `.to("cuda:0")`.

Pour plus d'informations sur comment utiliser les GPU sur colab, vous pouvez lire ce [tutoriel](https://colab.research.google.com/drive/1y3ZE4m-D7lPoMzsypSEXessYmjWfKGqD#scrollTo=3IEVK-KFxi5Z).


In [ ]:
use_gpu = torch.cuda.is_available()

print("GPU Available: {}".format(use_gpu))

# Le perceptron multi-couche
Un perceptron multi-couche est un réseau feed-forward simple. Il prend en entrée les images, les transforme à travers une série de couches cachées et finalement donne une sortie. Cette sortie correspond à la probabilité d'appartenance à l'une ou l'autre des classes de la cible.

Par exemple, si on regarde un perceptron multi-couche qui classifie des images de chiffres du jeu de données MNIST :

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/mlp.png?raw=true)

La procédure d'apprentissage typique pour ce modèle consiste en :
<ul>
<li>Définir l'architecture du réseau. Cela définira les paramètres (poids et biais) du réseau.</li>
<li>Définir la fonction de coût et l'optimiseur.</li>
<li>Entraîner le réseau.</li>
<li>Tester le réseau.</li>
</ul>

Notez que cette procèdure est valable pour l'entraînement de tous type de réseau de neurones profonds.

### Boîte à outils

Rappelons qu'un réseau de neurones profond peut être construit en utilisant la librairie <a href="http://pytorch.org/docs/master/nn.html">`torch.nn`</a>. `nn` travaille avec <a href="http://pytorch.org/docs/master/autograd.html">`torch.autograd`</a> pour définir et différencier les modèles.

## Définir l'architecture du réseau
### Boîte à outils

Pour définir l'architecture du réseau en pytorch il faut créer une classe enfant de la classe parent <a href="http://pytorch.org/docs/master/nn.html#torch.nn.Module">`torch.nn.Module`</a> où les méthodes suivantes sont à compléter :
<ul>
<li>La méthode `__init__` qui définit les couches. </li>
<li>La méthode `forward(input)` qui retourne l'`output`.</li>
</ul>

Pour construire les couches de l'`__init__` du perceptron multi-couche, les classes suivantes peuvent être utilisées :
<ul>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.Linear">`torch.nn.Linear(in_features, out_features)`</a> qui applique une transformation linéaire aux données d'entrée : y = Ax + b.</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.ReLU">`torch.nn.Relu()`</a> qui applique la fonction Relu éléments par éléments : Relu(x) = max(0, x).</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.Softmax">`torch.nn.Softmax(dim)`</a> qui applique la fonction Sofmax à un tenseur d'entrée à n-dimension en le normalisant de tel sorte que les éléments de tenseur de sortie à n-dimension soient dans l'intervalle [0, 1] et somment à 1.</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.Sequential">`torch.nn.sequential`</a>  qui est un conteneur séquentiel dans lequel les modules sont ajoutés dans l'ordre dans lequel ils sont passés au constructeur.</li>
</ul>

Dans `forward(input)` on applique aux données d'entrée les différentes couches définies dans `__init__` les unes après les autres.

Enfin, `model.to(device)` permet de passer le modèle sur GPU lorsque disponible.

## Implémentation

### Modèle

Nous voulons implémenter un modèle à trois couches avec la non-linéarité ReLU entre chaque couche.

In [ ]:
import torch.nn as nn
import copy

input_size = 784
hidden_size = 500
num_classes = 10

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()

        self.hidden_layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU())
        
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, num_classes))
    
    def forward(self, x):        
        
        out = self.hidden_layer(x)
        
        out = self.output_layer(out)
        
        return out

model = MLP(input_size, hidden_size, num_classes)
# switch model to GPU
model = model.to(device)

print(model)

print("\n\n# Parameters: ", sum([param.nelement() for param in model.parameters()]))

# Save the initial weights of model
init_model_wts = copy.deepcopy(model.state_dict())


### Nombre de paramètres

En entrée, nous avons 28x28 pixels applatits, donc un vecteur de taille 1x784. Afin de passer à une seconde couche de taille 500, nous avons besoin de 500 vecteurs de poids 1x784, donc 784\*500, ainsi que de 500 biais, donc 785\*500 total. Continuant cette logique, nous avons un nombre total de paramètres de:

`785*500 + 501*500 + 501*10 = 648010` 

## Définir la fonction de coût et l'optimiseur
### Boîte à outils
De nombreuses fonctions de coût et optimiseurs sont disponibles dans Pytorch. 

Rappelons qu'une fonction de coût $J(\theta) = L(x, y, \theta)$ prend en entrée le couple (prédiction, cible) et calcule une valeur qui estime la distance entre la prédiction et la cible. L'optimiseur dans le cas de la descente de gradient stochastique, ou Stochastic Gradient Descent (SGD), minimise la fonction de coût $J(\theta)$ paramétrisée par les poids du modèle $\theta \in \mathbb{R}^d$ en mettant à jour les poids itérativement suivant cette règle simple : `poids = poids - pas_d_apprentissage * gradient`.

Un choix commun pour un problème de classification (notre cas) est d'utiliser les classes suivantes :
<ul>
<li>**Fonction de coût :** <a href="http://pytorch.org/docs/master/nn.html#torch.nn.CrossEntropyLoss">`torch.nn.CrossEntropyLoss()`</a>. L'entropie croisée est souvent utilisée en optimisation. Elle permet de comparer une distribution $p$ avec une distibution de référence $t$. Elle est minimum lorsque $t=p$. Sa formule pour la calculer entre la prédiction et la cible est : $-\sum_j t_{ij} \log(p_{ij})$ où $p$ est la prédiction, $t$ la cible, $i$ les exemples et $j$ les classes de la cible. Note: la fonction <a href="http://pytorch.org/docs/master/nn.html#torch.nn.CrossEntropyLoss">`torch.nn.CrossEntropyLoss()`</a> calcule par défaut la valeur softmax de la dernière couche. </li>
<li>**Optimiseur :** <a href="http://pytorch.org/docs/master/optim.html#torch.optim.SGD">`torch.optim.SGD(net.parameters(), lr=learning_rate)`</a> qui est une implémentation de SGD.</li>
</ul>

### Implémentation

In [ ]:
learning_rate = 1e-2

criterion = nn.CrossEntropyLoss()  

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Entraîner le réseau
### Boîte à outils
En général, l'entraînement d'un réseau se fait en itérant sur plusieurs époques (une époque correspond à une passe sur l'intégralité du jeu de données d'entraînement). Sur une époque on va recevoir une série de batches fournies par l'itérateur. Pour chaque batch, on fait les opérations suivantes:
<ul>
<li>`optimizer.zero_grad()` : on efface les gradients encore stockés par le réseau issus de la passe précédente.</li>
<li>`loss.backward()` : on calcule automatiquement la dérivée du coût et on propage l'erreur dans le graphe par rétro-propagation.</li>
<li>`optimizer.step()` : on effectue une étape de descente de gradient. Dans le cas de SGD, c'est une descente de gradient classique avec les gradients calculés précédemment : `poids = poids - pas_d_apprentissage * gradient`.</li>
</ul>

### Implémentation

In [ ]:
import time
from torch.autograd import Variable

model.load_state_dict(init_model_wts)

since = time.time()

num_epochs = 10
train_loss_history = []
valid_loss_history = []

print("# Start training #")
for epoch in range(num_epochs):
    
    train_loss = 0
    train_n_iter = 0
    
    # Set model to train mode
    model.train()
    
    # Iterate over train data
    for images, labels in train_loader:  
        
        # put images on proper device (GPU)
        images = images.to(device)
        labels = labels.to(device)
        
        # Flatten the images
        images = images.view(-1, 28*28)

        # Zero the gradient buffer
        optimizer.zero_grad()  
        
        # Forward
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        
        # Optimize
        optimizer.step()
        
        # Statistics
        train_loss += loss.item()
        train_n_iter += 1
    
    valid_loss = 0
    valid_n_iter = 0
    
    # Set model to evaluate mode
    model.eval()
    
    # Iterate over valid data
    for images, labels in valid_loader:  
        
        # put images on proper device (GPU)
        images = images.to(device)
        labels = labels.to(device)
        
        # Flatten the images
        images = images.view(-1, 28*28)
        
        
        # Forward
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        # Statistics
        valid_loss += loss.item()
        valid_n_iter += 1
    
    train_loss_history.append(train_loss / train_n_iter)
    valid_loss_history.append(valid_loss / valid_n_iter)
    
    print('\nEpoch: {}/{}'.format(epoch + 1, num_epochs))
    print('\tTrain Loss: {:.4f}'.format(train_loss / train_n_iter))
    print('\tValid Loss: {:.4f}'.format(valid_loss / valid_n_iter))

time_elapsed = time.time() - since

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

Visualisons les courbes d'entraînement !

In [ ]:
# Save history for later
mlp_train_loss_history = train_loss_history
mlp_valid_loss_history = valid_loss_history

# Plot training and validation curve
x = range(1, num_epochs + 1)
plt.plot(x, mlp_train_loss_history, label='train')
plt.plot(x, mlp_valid_loss_history, label='valid')

plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Tester le réseau
### Boîte à outils
On évalue ensuite le réseau sur l'ensemble du jeu de données de test.
### Implémentation

In [ ]:
# Set model to evaluate mode
model.eval()

correct = 0
total = 0

# Iterate over test data
for images, labels in test_loader:
    
    # put images on proper device (GPU)
    images = images.to(device)
    labels = labels.to(device)
    
    # Flatten the images
    images = images.view(-1, 28*28)
    

    # Forward
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    
    # Statistics
    total += labels.size(0)
    correct += torch.sum(predicted == labels.data)

print('Accuracy on the test set: {:.2f}%'.format(100 * correct / total))

# Réseaux à convolutions




## Concepts

Nous revisitons ici certains concepts de base pour les réseaux à convolutions.

### Convolution

Une convolution consiste à prendre un filtre *k* et le faire "glisser" le long de l'entrée *I* afin d'obtenir une sortie *I*\**K*.

Un exemple de convolution 2D:

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/numerical_no_padding_no_strides.gif?raw=true)


### Filtres

Les filtres (kernel) sont utilisés afin d'extraire l'information utile des inputs. Ils ont généralement une taille *n* \* *n* avec *n* en général impair. La valeur des poids du filtre sont les paramètres qui seront appris par le réseau à convolution. 

Le filtre utilisé dans l'exemple de convolution précédent est:

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/d7acc4aeb74d9e9cb5fb51482a302196594837fe.png?raw=true)

### Profondeur

On utilise en général un nombre *M* de filtres qui correspondra à la profondeur de la couche. La profondeur en soit est un hyperparamètre du réseau. Ici, chaque filtre (cercles bleu), contribue à une couche de profondeur de l'image en sortie.

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/Conv_layer.png?raw=true)

### Pas

Le pas, ou *stride*, correspond à la taille du pas (mesuré en pixels) effectué lors de l'opération de la convolution. On emploie généralement un pas de 1 ou 2. Plus le pas est élevé, plus la dimension en sortie sera réduite.

### Marge à zéro

La marge à zéro, ou *zero padding*, consiste à ajouter une bordure de zéros autour du input. Ceci peut être utile, par exemple, lorsqu'on souhaite préserver la dimension de l'entrée à la sortie.

Voici un exemple de marge à zéro qui préserve la taille de l'entrée à la sortie. Ici, la marge à zéro=1, pas=1, et le filtre est de taille 3x3.

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/same_padding_no_strides.gif?raw=true)



### Max Pooling

Il est commun de retrouver dans les réseaux à convolutions des couches de *pooling*. Le but du pooling est de réduire la dimension des inputs entres couches de convolutions afin de réduire le nombre de paramètres nécessaires du réseau. Dans le cas de LeNet, nous nous servons d'un max pooling avec taille de filtre 2x2 et pas de 2. Ceci consiste à prendre la valeur maximale d'une région 2x2 puis de s'en servir comme input à la prochaine couche.

Voici un exemple de max pooling:

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/maxpool.jpeg?raw=true)


### Champ de vision

Le champ de vision, ou *Receptive Field*, est une mesure de la capacité du réseau à percevoir les détails à différentes échelles du input.  Dans un MLP, toutes les entrées sont connectés (fully connected), donc le champ de vision est la totalité de l'image. Dans le cas d'un réseau à convolution, plus il y a de filtres successifs, plus nous ajoutons au champ de vision. Considérons le cas simple d'un filtre 3x3 avec stride=1. Dans ce cas, pour un réseau à une couche, notre champ de vision est au maximum de 3x3. Cependant, plus nous ajoutons de couches, plus nous augmentons notre champ de vision. En ajoutant une seconde couche 3x3 avec pas de 1, notre champ de vision augmente a 5x5. En ajoutant une 3ème couche 3x3, notre champ de vision effectif est de 7x7. En utilisant 3 filtres 3x3 successifs, notre réseau necessite 3x3x3=27 paramètres, tandis qu'en utilisant 1 seul filtre de taille 7x7 requiert 49 paramètres. Il est donc plus optimal d'utliser plusieurs filtres successifs qu'un seul gros filtre pour un champ de vision équivalent. Également, en utilisant plusieurs filtres successifs, nous pouvons introduire plusieurs non-linéarités dans notre modèle.

Ici, le filtre de taille 3x3 (bordures grises) avec pas de 1 a un champ de vision de 5x5 (région jaune)

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/sNBmKMKAz-yJeCuS14usSqw.png?raw=true)


### Calculs de dimensions

De façon générale, pour une couche de convolution avec entrée de dimenion du volume
$W_1 * H_1 * D_1$ et avec les hyperparamètres

* Nombre de filtres = $K$

* Taille de filtre = $F$

* Taille de pas $S$

* Taille de marge $P$

Nous obtenons un volume en sortie de $W_2 * H_2 * D_2$ où 

* $W_2 = (W_1 - F + 2P) / S + 1$
* $H_2 = (H1 - F + 2P) / S + 1$
* $D_2 = K$

avec un nombre de paramètres total de $(F⋅F⋅D_1)⋅K$ poids et $K$ biais.

Pour une analyse plus détaillée, consultez [cet article](http://cs231n.github.io/convolutional-networks/)


## LeNet
LeNet est un réseau à convolution simple pour la classification. Il en existe plusieurs versions. Il est préférable d'utiliser un réseau à convolution pour de la classification d'images car ce type de réseau prend en compte la structure de l'image et à taille de réseau équivalent a un nombre de paramètres plus faible.

Par exemple, si on prend l'exemple de LeNet 5 pour classifier les images de chiffres du jeu de données MNIST :

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/lenet5.png?raw=true)


La procédure d'apprentissage typique pour ce modèle est la même que pour le perceptron multi-couche et consiste en :
<ul>
<li>Définir l'architecture du réseau. Cela définira les paramètres (poids et biais) du réseau.</li>
<li>Définir la fonction de coût et l'optimiseur.</li>
<li>Entraîner le réseau.</li>
<li>Tester le réseau.</li>
</ul>


## Définir l'architecture du réseau
### Boîte à outils
**Rappel :** Pour définir l'architecture du réseau en pytorch il faut créer une classe enfant de la classe parent <a href="http://pytorch.org/docs/master/nn.html#torch.nn.Module">`torch.nn.Module`</a> où les méthodes suivantes sont à compléter :
<ul>
<li>La méthode `__init__` qui définit les couches. </li>
<li>La méthode `forward(input)` qui retourne l'`output`.</li>
</ul>

Pour construire les couches de l'`__init__` de LeNet 5, les classes suivantes peuvent être utilisées :
<ul>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.Conv2d">`torch.nn.Conv2d(in_channels, out_channels, kernel_size)`</a> qui applique une convolution 2D sur un signal d'entrée composé de plusieurs canaux d'entrée.</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.MaxPool2d">`torch.nn.MaxPool2d(kernel_size)`</a> qui applique du max pooling 2D sur un signal d'entrée composé de plusieurs canaux d'entrée.</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.Linear">`torch.nn.Linear(in_features, out_features)`</a> qui applique une transformation linéaire aux données d'entrée : y = Ax + b.</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.ReLU">`torch.nn.Relu()`</a> qui applique la fonction Relu éléments par éléments : Relu(x) = max(0, x).</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.Softmax">`torch.nn.Softmax(dim)`</a> qui applique la fonction Sofmax à un tenseur d'entrée à n-dimension en le normalisant de tel sorte que les éléments de tenseur de sortie à n-dimension soient dans l'intervalle [0, 1] et somment à 1.</li>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.Sequential">`torch.nn.sequential`</a>  qui est un conteneur séquentiel dans lequel les modules sont ajoutés dans l'ordre dans lequel ils sont passés au constructeur.</li>
</ul>

Dans `forward(input)` on applique aux données d'entrée les différentes couches définies dans `__init__` les unes après les autres.
Il est aussi nécessaire de changer la forme des données juste avant l'appel à la couche linéaire, celle-ci n'acceptant que des données uni-dimensionelles. Il est possible d'utiliser <a href="https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view">`torch.Tensor.view(new_shape)`</a> pour cela.

Enfin, `model.to("cuda:0")` permet de passer le modèle sur GPU.

### Implémentation

In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        
        # À compléter
    def forward(self, x):
        
        # À compléter
        
        return out
        
model = LeNet5()
model = model.to(device)

print(model)
print("\n\n# Parameters: ", sum([param.nelement() for param in model.parameters()]))

 Ici, on observe 28 938 paramètres pour LeNet5 contre 648 010 paramètres pour le MLP à deux couches cachées. On a donc une réduction significative du nombre de paramètres entre LeNet5 et le MLP précédent. Le calcul du nombre de paramètres se résume ainsi:
 


```
1ère couche: 16 filtres de taille 5x5 + 16 biais = 16*5*5 + 16 = 416
2ème couche: 16 * 32 filtres de taille 5x5 + 32 biais = 16*32*5*5 + 32 = 12 832
Couche FC: 7*7*32*10 + 10 biais = 15 690
 
Total = 416 + 12 832 + 15 690 = 28 938
```



In [ ]:
# Save the initial weights of model
init_model_wts = copy.deepcopy(model.state_dict())

## Définir la fonction de coût et l'optimiseur
### Boîte à outils
**Rappel : ** un choix commun pour un problème de classification (notre cas) est d'utiliser les classes suivantes :
<ul>
<li>**Fonction de coût :** <a href="http://pytorch.org/docs/master/nn.html#torch.nn.CrossEntropyLoss">`torch.nn.CrossEntropyLoss()`</a>. L'entropie croisée est souvent utilisée en optimisation. Elle permet de comparer une distribution $p$ avec une distibution de référence $t$. Elle est minimum lorsque $t=p$. Sa formule pour la calculer entre la prédiction et la cible est : $-\sum_j t_{ij} \log(p_{ij})$ où $p$ est la prédiction, $t$ la cible, $i$ les exemples et $j$ les classes de la cible.</li>
<li>**Optimiseur :** <a href="http://pytorch.org/docs/master/optim.html#torch.optim.SGD">`torch.optim.SGD(net.parameters(), lr=learning_rate)`</a> qui est une implémentation de SGD.</li>
</ul>

### Implémentation


In [ ]:
# À compléter

criterion = ...
optimizer = ...

## Entraîner le réseau
### Boîte à outils
**Rappel :** en général, l'entraînement d'un réseau se fait en itérant sur plusieurs époques (une époque correspond à une passe sur l'intégralité du jeu de données d'entraînement). Sur une époque on va recevoir une série de batches fournies par l'itérateur. Pour chaque batch, on fait les opérations suivantes:
<ul>
<li>`optimizer.zero_grad()` : on efface les gradients encore stockés par le réseau issus de la passe précédente.</li>
<li>`loss.backward()` : on calcule automatiquement la dérivée du coût et on propage l'erreur dans le graphe par rétro-propagation.</li>
<li>`optimizer.step()` : on effectue une étape de descente de gradient. Dans le cas de SGD, c'est une descente de gradient classique avec les gradients calculés précédemment : `poids = poids - pas_d_apprentissage * gradient`.</li>
</ul>

### Implémentation
C'est simple, compléter les trous...

Bonne chance !

In [ ]:
model.load_state_dict(init_model_wts)

since = time.time()

num_epochs = 10
train_loss_history = []
valid_loss_history = []

print("# Start training #")
for epoch in range(num_epochs):
    
    train_loss = 0
    train_n_iter = 0
    
    # Set model to train mode
    model.train()
    
    # Iterate over train data
    for images, labels in train_loader:  

        # put images on proper device (GPU)
        images = ...
        labels = ...
        
        # Zero the gradient buffer
        ...
        
        # Forward pass
        outputs = ...
        
        # Calculate loss
        loss = criterion(...)
        
        # Backward pass
        ...
        
        # Optimize
        ...
        
        # Statistics
        train_loss += loss.item()
        train_n_iter += 1
    
    valid_loss = 0
    valid_n_iter = 0
    
    # Set model to evaluate mode
    model.eval()
    
    # Iterate over valid data
    for images, labels in valid_loader:  
        
        # put images on proper device (GPU)
        images = ...
        labels = ...
        
        # Forward pass
        outputs = ...
        
        # Calculate loss
        loss = criterion(...)
        
        # Statistics
        valid_loss += loss.item()
        valid_n_iter += 1
    
    train_loss_history.append(train_loss / train_n_iter)
    valid_loss_history.append(valid_loss / valid_n_iter)
    
    print('\nEpoch: {}/{}'.format(epoch + 1, num_epochs))
    print('\tTrain Loss: {:.4f}'.format(train_loss / train_n_iter))
    print('\tValid Loss: {:.4f}'.format(valid_loss / valid_n_iter))

time_elapsed = time.time() - since

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

Visualisons les courbes d'entraînement !

In [ ]:
# Save history for later
lenet5_train_loss_history = train_loss_history
lenet5_valid_loss_history = valid_loss_history

# Plot training and validation curve
x = range(1, num_epochs + 1)
plt.plot(x, lenet5_train_loss_history, label='train')
plt.plot(x, lenet5_valid_loss_history, label='valid')

plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

On peut superposer les courbes d'entraînement et de validation de LeNet5 et du MLP :

In [ ]:
# Plot training and validation curve
x = range(1, num_epochs + 1)
plt.plot(x, mlp_train_loss_history, label='MLP train')
plt.plot(x, mlp_valid_loss_history, label='MLP valid')
plt.plot(x, lenet5_train_loss_history, label='LeNet5 train')
plt.plot(x, lenet5_valid_loss_history, label='LeNet5 valid')

plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Tester le réseau
### Boîte à outils
**Rappel :** on évalue ensuite le réseau sur l'ensemble du jeu de données de test.
### Implémentation

In [ ]:
# Set model to evaluate mode
model.eval()

correct = 0
total = 0

# Iterate over data.
for images, labels in test_loader:
    
    # put images on proper device (GPU)
    images = ...
    labels = ...
    
    # No need to flatten the images here !
    

    # Forward
    outputs = ...
    _, predicted = torch.max(outputs.data, 1)
    
    # Statistics
    total += labels.size(0)
    correct += ...

print('Accuracy on the test set: {:.2f}%'.format(100 * correct / total))

On obtient de meilleurs résultats après 10 époques !

---



## Méthodes pratiques pour améliorer l'entraînement

### Batch normalization
Le *batch normalization* est une astuce qui permet, en pratique, au modèle d'apprendre plus vite. Elle agit comme régularisateur en normalisant les entrées par batch, de manière différentiable. Les données en sorties de cette couche auront une moyenne proche de 0 et une variance proche de 1.

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/1_Hiq-rLFGDpESpr8QNsJ1jg.png?raw=true)


Pour plus d'informations sur le *Batch Normalization*, consultez cet [article](https://arxiv.org/pdf/1502.03167v3.pdf).

### Boîte à outils
Pour ajouter la batch normalisation dans LeNet5, il suffit de l'ajouter parmis les couches de l'`__init__` et de l'appeler après chaque couche de convolution. La classe suivante peut être utilisée:
<ul>
<li><a href="http://pytorch.org/docs/master/nn.html#torch.nn.BatchNorm2d">`nn.BatchNorm2d(num_features)`</a> : permet d'ajouter de la batch normalisation à une entrée à 4 dimensions présentée sous la forme d'un tenseur à 3 dimensions.</li>
</ul>

### Implémentation


In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # À compléter
        
    def forward(self, x):
        # À compléter
        
        return out
        
model = LeNet5()
model = model.to(device)
  
print(model)

print("\n\n# Parameters: ", sum([param.nelement() for param in model.parameters()]))

Ici, on observe 29 034 paramètres pour LeNet5 avec batch normalisation contre 28 938 paramètres pour LeNet5 sans batch normalisation.

In [ ]:
# Save the initial weights of model
init_model_wts = copy.deepcopy(model.state_dict())

**L'implémentation de la fonction de coût, l'optimiseur, les boucles d'entraînement et de test du réseau reste inchangé !**

In [ ]:
criterion = ...
optimizer = ...
model.load_state_dict(init_model_wts)

since = time.time()

num_epochs = 10
train_loss_history = []
valid_loss_history = []

print("# Start training #")
for epoch in range(num_epochs):
    
    train_loss = 0
    train_n_iter = 0
    
    # Set model to train mode
    model.train()
    
    # Iterate over train data
    for images, labels in train_loader:  

        # put images on proper device (GPU)
        images = ...
        labels = ...
        
        # Zero the gradient buffer
        ...
        
        # Forward pass
        outputs = ...
        
        # Calculate loss
        loss = criterion(...)
        
        # Backward pass
        ...
        
        # Optimize
        ...
        
        # Statistics
        train_loss += loss.item()
        train_n_iter += 1
    
    valid_loss = 0
    valid_n_iter = 0
    
    # Set model to evaluate mode
    model.eval()
    
    # Iterate over valid data
    for images, labels in valid_loader:  
        
        # put images on proper device (GPU)
        images = ...
        labels = ...
        
        # Forward pass
        outputs = ...
        
        # Calculate loss
        loss = criterion(...)
        
        # Statistics
        valid_loss += loss.item()
        valid_n_iter += 1
    
    train_loss_history.append(train_loss / train_n_iter)
    valid_loss_history.append(valid_loss / valid_n_iter)
    
    print('\nEpoch: {}/{}'.format(epoch + 1, num_epochs))
    print('\tTrain Loss: {:.4f}'.format(train_loss / train_n_iter))
    print('\tValid Loss: {:.4f}'.format(valid_loss / valid_n_iter))

time_elapsed = time.time() - since

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

On obtient d'encore meilleurs résultats après 10 époques !

Regardons les coubres d'entraînement et de validation :

In [ ]:
# Save history for later
lenet5_batchnorm_train_loss_history = train_loss_history
lenet5_batchnorm_valid_loss_history = valid_loss_history

# Plot training and validation curve
x = range(1, num_epochs + 1)
plt.plot(x, lenet5_train_loss_history, label='LeNet5 train')
plt.plot(x, lenet5_valid_loss_history, label='LeNet5 valid')
plt.plot(x, lenet5_batchnorm_train_loss_history, label='LeNet5 batch norm train')
plt.plot(x, lenet5_batchnorm_valid_loss_history, label='LeNet5 batch norm valid')

plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Transfer Learning : finetuning d'un réseau à convolution
**Attribution :** cette partie reprend en partie le tutoriel : http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

En pratique, il est peu commun d'entraîner un réseau à convolution à partir de rien (c'est-à-dire avec une initialisation des poids aléatoires). En effet, souvent, le jeu de données d'intérêt est trop petit. A la place, il est commun de pré-entraîner le réseau sur un jeu de données plus gros comme, par exemple, un sous-ensemble d'ImageNet (1.2 millions d'images avec 1000 catégories). Ce réseau pré-entraîné est ensuite utilisé comme initialisation des poids du réseau qui sera entraîné sur le jeu de données d'intérêt. On parle de finetuning du réseau à convolution. A noter que le réseau pré-entraîné peut aussi être utilisé pour extraire de nouvelles variables du jeu de données d'intérêt. On parle de transfer learning.

Nous allons maintenant étudier plus en détail le scénario du finetuning pour le transfer learning.

## Télécharger les données et créer le chargeur de données
Le jeu de données que nous allons étudier est un sous-ensemble d'ImageNet qui contient environ $120 \times 2$ images d'entraînement et $75 \times 2$ images de test de fourmis et d'abeilles. Le but est de classifier ces deux classes. Ci-dessous, un exemples d'images de ce jeu de données :


![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/fourmi_abeille.png?raw=true)

### Boîte à outils
**Rappel :** une façon simple de charger les données dans PyTorch est : 
<ul>
<li>D'utiliser une classe enfant de la classe parent <a href="http://pytorch.org/docs/master/data.html#torch.utils.data.Dataset">`torch.utils.data.Dataset`</a> où les méthodes `__getitem__` et `__len__` sont à compléter. Notez qu' à ce stade, les données ne sont pas chargées en mémoire.</li>
<li>D'utiliser la classe <a href="http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader">`torch.utils.data.DataLoader`</a> pour lire et mettre en mémoire les données.</li>
</ul>

**Remarque :** <a href="http://pytorch.org/docs/master/torchvision/datasets.html#torchvision-datasets">`torchvision.datasets`</a> peut aussi être utilisé pour charger des données à partir d'un dossier.

**Augmentation des données :** pour augmenter les données, <a href="http://pytorch.org/docs/master/torchvision/transforms.html#torchvision-transforms">`torchvision.transforms`</a> fournit les transformations d'images courantes. Ces transformations peuvent être appliquées successivement en utilisant la classe <a href="http://pytorch.org/docs/master/torchvision/transforms.html#torchvision.transforms.Compose">`torchvision.transforms.Compose`</a>.

### Implémentation

In [ ]:
%%bash
## DOWNLOAD DATASET ##
if [ ! -d "hymenoptera_data" ]; then
  wget --quiet https://download.pytorch.org/tutorial/hymenoptera_data.zip
  unzip -q hymenoptera_data.zip
  rm hymenoptera_data.zip
fi

In [ ]:
import os
from PIL import Image
from torchvision import datasets


def make_dataset(root, split_type):
    """
    Parameters
    ----------
    root_dir : string
    Directory with all the images.
    split_type : string
    The name of the split in {'train', 'valid'}.

    Returns
    -------
    images : dict
    Dict of images path for each classes for a specific split type.
    """

    images = {}
    root = os.path.join(root, split_type)

    for classes in sorted(os.listdir(root)):
        images[classes] = []
    path_classes = os.path.join(root, classes)

    for root_, _, fnames in sorted(os.walk(path_classes)):
        for fname in sorted(fnames):
            if fname.endswith('.jpg'):
                item = os.path.join(root_, fname)
                images[classes].append(item)

    return images


class HymenopteraDataset(torch.utils.data.Dataset):
    """Hymenoptera dataset."""

    def __init__(self, root_dir, split_type='train', transform=None):
        """
        Parameters
        ----------
        root_dir : string
           Directory with all the images.
        split_type : string
           The name of the split in {'train', 'valid', 'test', 'train_valid'}.
        transform : callable, optional
           Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.split_type = split_type
        self.transform = transform
        self.classes = {'ants': 0, 'bees': 1}

        imgs_ = []
        target_ = []

        if split_type == 'train':
            imgs = make_dataset(root_dir, 'train')
            for k, v in imgs.items():
                imgs_ += imgs[k][:int(0.8*len(v))]
                target_ += [self.classes[k]] * len(imgs_)

        elif split_type == 'valid':
            imgs = make_dataset(root_dir, 'train')
            for k, v in imgs.items():
                imgs_ += imgs[k][int(0.8*len(v)):]
                target_ += [self.classes[k]] * len(imgs_)

        elif split_type == 'train_valid':
            imgs = make_dataset(root_dir, 'train')
            for k, v in imgs.items():
                imgs_ += imgs[k]
                target_ += [self.classes[k]] * len(imgs_)

        elif split_type == 'test':
            imgs = make_dataset(root_dir, 'val')
            for k, v in imgs.items():
                imgs_ += imgs[k]
                target_ += [self.classes[k]] * len(imgs_)

        self.imgs = imgs_
        self.target = np.array(target_)

    def __len__(self):
        """Get the number of image in the dataset.

        Returns
        -------
        int
           The number of images in the dataset.
        """
        return len(self.imgs)

    def __getitem__(self, index):
        """Get the items : image, target

        Parameters
        ----------
        index : int
           Index

        Returns
        -------
        img : tensor
           The image.
        target : int
           Target is class_index of the target class.
        """
        path = self.imgs[index]
        target = self.target[index]

        with open(path, 'rb') as f:
            with Image.open(f) as img:
                img.convert('RGB')

                if self.transform:
                    img = self.transform(img)

        return img, target


## Augmentation des données 
Une astuce souvent utilisée pour éviter l'overfit et augmenter la taille apparente du jeu de données d'entrainement est l'augmentation de données. Il s'agit de différents types de transformations aux inputs afin de les déformer légèrement les images. Par exemple, les images peuvent être agrandies, élargies, tournées sur elle mêmes, miroitées, etc. Voici un exemple de différentes augmentations:

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/1_Jujct_Pt-zvdWtSFpHUp3Q.png?raw=true)


Pour augmenter les données, <a href="http://pytorch.org/docs/master/torchvision/transforms.html#torchvision-transforms">`torchvision.transforms`</a> fournit les transformations d'images courantes. Ces transformations peuvent être appliquées successivement en utilisant la classe <a href="http://pytorch.org/docs/master/torchvision/transforms.html#torchvision.transforms.Compose">`torchvision.transforms.Compose`</a>.

Ajoutez les transformations suivantes pour l'ensemble d'entraînement:
* Un crop aléatoire qui redimensionne l'image à une taille 224x224
* Une probabilité aléatoire de prendre une copie miroitée de l'image
* Une normalisation avec les valeurs suivantes pour les moyennes et écarts types: [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

Pour l'ensemble de validation, redimensionnez l'image à 256x256, prenez un crop à partir du centre de l'image et normalisez l'image avec les mêmes valeurs que pour l'ensemble d'entraînement


In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        # À compléter    
    ]),
    'valid': transforms.Compose([
        # À compléter
    ]),
}

In [ ]:
# Dataset loader
data_dir = 'hymenoptera_data'

data_train = HymenopteraDataset(data_dir, 'train', data_transforms['train'])
train_loader = DataLoader(data_train, batch_size=4, shuffle=True, num_workers=4)

data_valid = HymenopteraDataset(data_dir, 'valid', data_transforms['valid'])
valid_loader = DataLoader(data_valid, batch_size=4, shuffle=False, num_workers=4)

data_test = HymenopteraDataset(data_dir, 'test', data_transforms['valid'])
test_loader = DataLoader(data_test, batch_size=4, shuffle=False, num_workers=4)

print('# images in data train: {}'.format(len(data_train)))
print('# images in data valid: {}'.format(len(data_valid)))
print('# images in data test: {}'.format(len(data_test)))

Visualisons les données d'entraînement !

In [ ]:
inputs, classes = next(iter(train_loader))

print('Classes: {}'.format(data_train.classes))
print('Inputs size: {}'.format(inputs.size()))
print('Classes size: {}'.format(classes.size()))

# Random image of the batch
idx = np.random.randint(len(inputs))
img = inputs[idx]
labels = list(data_train.classes.keys())
img_label = labels[(classes[idx])]

img = img.numpy().transpose((1, 2, 0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img = std * img + mean
img = np.clip(img, 0, 1)
plt.imshow(img)
plt.title(img_label)
plt.grid(False)
plt.show()

In [ ]:
import torchvision

def imshow(img, title=None):
    """Imshow for Tensor."""
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.grid(False)
    plt.show()

inputs, classes = next(iter(train_loader))

out = torchvision.utils.make_grid(inputs)

class_names = data_train.classes
class_names = {class_names[k]: k for k in class_names.keys()}

imshow(out, title=[class_names[int(x)] for x in classes])

## Définir l'architecture du réseau
### Boîte à outils
Ici, nous voulons réutiliser un réseau pré-entrainé sur ImageNet. Pour cela, il faut charger un modèle pré-entraîné et réinitialiser la couche finale qui est la couche complètement connectée. Par chance, dans Pytorch, <a href="http://pytorch.org/docs/0.1.12/torchvision/models.html#module-torchvision.models">`torchvision.models`</a> propose des architectures toutes faites où les poids ont déjà été entraînés sur ImageNet.

Un choix commun pour un problème de classification (notre cas) est d'utiliser le modèle *ResNet18*. Pour plus d'informations sur cette architecture, consultez cet [article](https://arxiv.org/abs/1512.03385). Vous pouvez également consulter la documentation PyTorch du modèle:
<a href="http://pytorch.org/docs/0.1.12/torchvision/models.html#torchvision.models.resnet18">`torchvision.models.resnet18(pretrained=True)`</a>

Un exemple de bloc résiduel est donné ci-dessous :

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/bloc_residuel.png?raw=true)


**Rappel :** <a href="http://pytorch.org/docs/master/nn.html#torch.nn.Linear">`torch.nn.Linear(in_features, out_features)`</a> permet d'appliquer une transformation linéaire à des données d'entrée : y = Ax + b.

### Implémentation

Dans un premier temps on va utiliser un modèle non pré-entraîné puis on utilisera le même modèle mais cette fois-ci pré-entraîné afin de comparer les résultats.

In [ ]:
from torchvision import models

# Load non-pre-trained resnet18 model
model = models.resnet18(pretrained=False)

# Reset last layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

print(model)

print("\n\n# Parameters: ", sum([param.nelement() for param in model.parameters()]))

In [ ]:
# Save the initial weights of model
init_model_wts = copy.deepcopy(model.state_dict())

## Définir la fonction de coût et l'optimiseur
### Boîte à outils
**Rappel : ** un choix commun pour un problème de classification (notre cas) est d'utiliser les classes suivantes :
<ul>
<li>**Fonction de coût :** <a href="http://pytorch.org/docs/master/nn.html#torch.nn.CrossEntropyLoss">`torch.nn.CrossEntropyLoss()`</a>. L'entropie croisée est souvent utilisée en optimisation. Elle permet de comparer une distribution $p$ avec une distibution de référence $t$. Elle est minimum lorsque $t=p$. Sa formule pour la calculer entre la prédiction et la cible est : $-\sum_j t_{ij} \log(p_{ij})$ où $p$ est la prédiction, $t$ la cible, $i$ les exemples et $j$ les classes de la cible.</li>
<li>**Optimiseur :** <a href="http://pytorch.org/docs/master/optim.html#torch.optim.SGD">`torch.optim.SGD(net.parameters(), lr=learning_rate)`</a>. Utilisez un taux d'apprentissage de 1e-3 et une valeur de momentum de 0.9</li>
</ul>

### Implémentation

In [ ]:
criterion = ...

learning_rate = 1e-3
optimizer = ...

## Entraîner le réseau
### Boîte à outils
**Rappel :** en général, l'entraînement d'un réseau se fait en itérant sur plusieurs époques (une époque correspond à une passe sur l'intégralité du jeu de données d'entraînement). Sur une époque on va recevoir une série de batches fournies par l'itérateur. Pour chaque batch, on fait les opérations suivantes:
<ul>
<li>`optimizer.zero_grad()` : on efface les gradients encore stockés par le réseau issus de la passe précédente.</li>
<li>`loss.backward()` : on calcule automatiquement la dérivée du coût et on propage l'erreur dans le graphe par rétro-propagation.</li>
<li>`optimizer.step()` : on effectue une étape de descente de gradient. Dans le cas de SGD, c'est une descente de gradient classique avec les gradients calculés précédemment : `poids = poids - pas_d_apprentissage * gradient`. Dans le cas d'Adam une opération légérement plus complexe est réalisée.</li>
</ul>

**Conseils bonus :** L'orsque l'on entraîne le réseau de neurones profonds, il est conseillé de faire :
<ul>
<li>de l'early stopping. C'est une forme de régularisation qui évite de faire du sur-apprentissage en utilisant une règle pour stopper l'apprentissage du modèle.</li>
<li>du checkpointing. Pour cela, il est commun d'enregister les poids du réseau accessible avec `model.state_dict()` à différentes étapes de l'entraînement.</li>
<li>d'imprimer les temps d'exécution. Pour cela, il est commun d'utilier `time.time()`.</li>
</ul>

### Implémentation

In [ ]:
import time

since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())

num_epochs = 25
best_acc = 0.0

train_loss_history = []
valid_loss_history = []

print("# Start training #")
for epoch in range(num_epochs):
    
    train_loss = 0
    train_n_iter = 0
    
    # Set model to train mode
    ...
    
    # Iterate over train data
    for images, labels in train_loader:  
        
        # put images on proper device (GPU)
        images = ...
        labels = ...

        # Zero the gradient buffer
        ...
        
        # Forward Pass
        outputs = ...
        
        loss = criterion(...)
        
        # Backward Pass
        ...
        
        # Optimize
        ...
        
        # Statistics
        train_loss += loss.item()
        train_n_iter += 1
    
    valid_loss = 0
    valid_n_iter = 0
    
    # Set model to evaluate mode
    ...
    
    # Iterate over valid data
    total = 0
    correct = 0
    for images, labels in valid_loader:  
        
        # put images on proper device (GPU)
        images = ...
        labels = ...
        
        # Forward
        outputs = ...
        _, predicted = torch.max(outputs.data, 1)
        
        loss = criterion(outputs, labels)
    
        # Statistics
        total += labels.size(0)
        correct += torch.sum(predicted == labels.data)
        valid_loss += loss.item()
        valid_n_iter += 1
    
    epoch_acc = 100 * correct / total
    
    # Deep copy the best model
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
    
    train_loss_history.append(train_loss / train_n_iter)
    valid_loss_history.append(valid_loss / valid_n_iter)
    
    print('\nEpoch: {}/{}'.format(epoch + 1, num_epochs))
    print('\tTrain Loss: {:.4f}'.format(train_loss / train_n_iter))
    print('\tValid Loss: {:.4f}'.format(valid_loss / valid_n_iter))

time_elapsed = time.time() - since

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

print('\n\nBest valid accuracy: {:.2f}'.format(best_acc))

Visualisons les courbes d'entraînement et de validation :

In [ ]:
resnet18_train_loss_history = train_loss_history
resnet18_valid_loss_history = valid_loss_history

# Plot training and validation curve
x = range(1, num_epochs + 1)
plt.plot(x, resnet18_train_loss_history, label='ResNet18 train')
plt.plot(x, resnet18_valid_loss_history, label='ResNet18 valid')

plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Tester le réseau
### Boîte à outils
**Rappel :** on évalue ensuite le réseau sur l'ensemble du jeu de données de test.

**Remarque :** ici, nous n'avons pas de données de test donc nous testons sur l'ensemble de validation (à ne pas faire en pratique).

**Utilisation des poids du meilleur modèle :** comme nous avons fait de l'early stopping lors de l'entraînement, nous voulons réutiliser les poids du meilleur modèle sur l'ensemble de validation pour tester le modèle. Ces poids ont été enregistrés lors de l'entraînement du modèle dans `best_model_wts`. Pour les charger il suffit d'utiliser `model.load_state_dict(best_model_wts)`.
### Implémentation

In [ ]:
# Load best model weights
model.load_state_dict(best_model_wts)

# Set model to evaluate mode
model.eval()

correct = 0
total = 0

# Iterate over test data
for images, labels in test_loader:
    
    # put images on proper device (GPU)
    images = ...
    labels = ...
    
    # Forward
    outputs = ...
    _, predicted = torch.max(outputs.data, 1)
    
    # Statistics
    total += labels.size(0)
    correct += torch.sum(predicted == labels.data)

print('Accuracy on the test set: {:.2f}%'.format(100 * correct / total))

Avec les poids pré-entraînés :

In [ ]:
from torchvision import models

# Load pre-trained resnet18 model
model = models.resnet18(pretrained=True)

# Reset last layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

print(model)

print("\n\n# Parameters: ", sum([param.nelement() for param in model.parameters()]))

In [ ]:
# Save the initial weights of model
init_model_wts = copy.deepcopy(model.state_dict())

In [ ]:
criterion = ...
optimizer = ...

In [ ]:
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())

num_epochs = 25
best_acc = 0.0

train_loss_history = []
valid_loss_history = []

print("# Start training #")
for epoch in range(num_epochs):
    
    train_loss = 0
    train_n_iter = 0
    
    # Set model to train mode
    ...
    
    # Iterate over train data
    for images, labels in train_loader:  
        
        # put images on proper device (GPU)
        images = ...
        labels = ...

        # Zero the gradient buffer
        ...
        
        # Forward pass
        outputs = ...
        
        loss = ...
        
        # Backward pass
        ...
        
        # Optimize
        ...
        
        # Statistics
        train_loss += loss.item()
        train_n_iter += 1
    
    valid_loss = 0
    valid_n_iter = 0
    
    # Set model to evaluate mode
    model.eval()
    
    # Iterate over valid data
    total = 0
    correct = 0
    for images, labels in valid_loader:  
        
        # put images on proper device (GPU)
        images = ...
        labels = ...
        
        # Forward
        outputs = ...
        _, predicted = torch.max(outputs.data, 1)
        
        loss = criterion(outputs, labels)
    
        # Statistics
        total += labels.size(0)
        correct += torch.sum(predicted == labels.data)
        valid_loss += loss.item()
        valid_n_iter += 1
    
    epoch_acc = 100 * correct / total
    
    # Deep copy the best model
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
    
    train_loss_history.append(train_loss / train_n_iter)
    valid_loss_history.append(valid_loss / valid_n_iter)
    
    print('\nEpoch: {}/{}'.format(epoch + 1, num_epochs))
    print('\tTrain Loss: {:.4f}'.format(train_loss / train_n_iter))
    print('\tValid Loss: {:.4f}'.format(valid_loss / valid_n_iter))

time_elapsed = time.time() - since

print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

print('\n\nBest valid accuracy: {:.2f}'.format(best_acc))

Visualisons les courbes d'entraînement et de validation :

In [ ]:
resnet18_pretrained_train_loss_history = train_loss_history
resnet18_pretrained_valid_loss_history = valid_loss_history

# Plot training and validation curve
x = range(1, num_epochs + 1)
plt.plot(x, resnet18_train_loss_history, label='ResNet18 train')
plt.plot(x, resnet18_valid_loss_history, label='ResNet18 valid')
plt.plot(
      x, resnet18_pretrained_train_loss_history,
    label='ResNet18 pretrained train')
plt.plot(
      x, resnet18_pretrained_valid_loss_history,
    label='ResNet18 pretrained valid')

plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

Testons le modèle :

In [ ]:
# Load best model weights
model.load_state_dict(best_model_wts)

# Set model to evaluate mode
model.eval()

correct = 0
total = 0

# Iterate over test data
for images, labels in test_loader:
    
    # put images on proper device (GPU)
    images = ...
    labels = ...
    
    # Forward pass
    outputs = ...
    _, predicted = torch.max(outputs.data, 1)
    
    # Statistics
    total += labels.size(0)
    correct += torch.sum(predicted == labels.data)

print('Accuracy on the test set: {:.2f}%'.format(100 * correct / total))

On observe une augmentation de l'accuracy sur le test par rapport au modèle dont les poids n'avaient pas été entraîné.

# Suite (optionnelle)

Si vous souhaitez approfondir vos connaissances sur les réseaux à convolution, voici une série de liens vers divers méthodes traitant sur des applications plus avancées.

## Segmentation d'image

Bien qu'il soit pratique de pouvoir classifier une image, il est encore plus pratique de pouvoir classifier quelle partie de l'image contient la catégorie en question par pixel. Il s'agit de la segmentation d'image. Un exemple populaire est l'algorithme [Mask R-CNN](https://arxiv.org/abs/1703.06870). Vous pouvez trouver un tutoriel sur l'implémentation de cet algorithme [ici].(https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/train_shapes.ipynb)

Voici un exemple d'image segmentée par catégrorie:

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/detection_final.png?raw=true)


## Modèles génératifs (exemple des GANs)

Un GAN consiste de deux réseaux qui se font concurrence. Ils ont pour but de se surpasser l'un l'autre. Un réseau génère des images fictives et tente de convaincre l'autre réseau du réalisme de l'image tandis que l'autre réseau est entrainé à discerner une vraie image d'une fausse. Ceci permet la générations d'images réalistes lorsque les deux réseaux sont bien entrainés. Vous pouvez trouver la publication originale [ici](http://papers.nips.cc/paper/5423-generative-adversarial-nets) ainsi qu'une implémentation [ici](https://github.com/diegoalejogm/gans)

Voici un exemple de GAN sur MNIST: 

![Alt Text](https://github.com/mila-iqia/ecole_dl_mila_ivado/blob/master/tutoriaux/CNN/images/1_nAVqFluPijpBWR2tI4gCxg.png?raw=true)


# Références
Certaines parties de ce tutoriel sont fortement inspirées des tutoriaux suivant :
<ul>
<li>https://github.com/andrewliao11/dni.pytorch/blob/master/mlp.py
<li>https://github.com/andrewliao11/dni.pytorch/blob/master/cnn.py
<li>http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
<li>http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
 <li>http://cs231n.github.io/convolutional-networks/
 <li>http://deeplearning.net/software/theano/tutorial/conv_arithmetic.html#convolution-as-a-matrix-operation
</ul>